In [62]:
import numpy as np
import pandas as pd
import os
# Lista de posibles rutas
addresses = [
    'C:/Users/RONALD Q/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/IEXCITACION.xlsx',
    'C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/IEXCITACION.xlsx',
    'C:/Users/mticllacu/OneDrive - LUZ DEL SUR S.A.A/Archivos de Ronald Quispe Ocaña - ProyectoRyD_V2/Basededatos/IEXCITACION.xlsx'
]

df = None
for path in addresses:
    if os.path.exists(path):   # verifica si existe
        df = pd.read_excel(path,header=1)
        print(f"✅ Archivo cargado desde: {path}")
        break

if df is None:
    raise FileNotFoundError("❌ No se encontró el archivo en ninguna de las rutas especificadas.")

✅ Archivo cargado desde: C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/IEXCITACION.xlsx


In [63]:
df = df.drop(columns=['Unnamed: 0'])
df = df.rename(columns ={'FECHA DE MUESTRA':'FECHA'})
df.head()

,SERIE,FECHA,R POS1 mA,R POS2 mA,R POS3 mA,R POS4 mA,R POS5 mA,R POS6 mA,R POS7 mA,R POS8 mA,...,T POS18 mA,T POS19 mA,T POS20 mA,T POS21 mA,T POS22 mA,T POS23 mA,T POS24 mA,T POS25 mA,T POS26 mA,T POS27 mA
0,146660T3,2022-05-12,111.235,191.629,146.622,144.914,179.458,121.242,188.996,197.978,...,186.193,160.812,120.611,114.643,143.999,132.651,176.578,161.892,147.003,124.753
1,146660T3,2018-02-16,188.693,114.216,133.965,194.859,160.695,145.281,110.364,116.006,...,152.812,194.145,129.906,163.382,188.661,107.555,156.040,119.796,104.434,170.016
2,146660T3,2017-04-15,100.873,100.875,110.410,124.139,189.568,145.275,110.087,175.354,...,155.308,115.643,121.963,182.579,143.042,164.656,102.757,142.567,173.857,118.402
3,146660T3,2016-09-24,128.112,143.145,103.937,109.001,169.662,195.572,117.368,198.749,...,179.454,101.771,155.218,146.779,164.411,111.809,188.801,148.031,146.680,110.143
4,364076,2022-06-18,142.290,180.712,173.369,194.856,119.309,142.324,199.955,163.990,...,163.629,121.602,137.777,198.430,153.521,186.329,136.476,145.228,190.737,160.036


In [64]:
# Calcular columnas delta respecto al valor inicial por SERIE en la fecha mínima
# Detectar columnas numéricas a procesar (excluyendo SERIE y FECHA)
cols_val = [c for c in df.columns if c not in ['SERIE', 'FECHA'] and np.issubdtype(df[c].dtype, np.number)]

for col in cols_val:
    ref = df.groupby('SERIE').apply(lambda g: g.loc[g['FECHA'].idxmin(), col])
    ref_mapped = df['SERIE'].map(ref)
    delta = abs((df[col] - ref_mapped) / ref_mapped) * 100
    delta = delta.replace([np.inf, -np.inf], np.nan)
    mask = df[col].isna() | ref_mapped.isna() | (ref_mapped == 0) | (df[col] == 0)
    delta[mask] = np.nan
    df[f'{col}_Delta'] = delta
df

C:\Users\roquispec\AppData\Local\Temp\ipykernel_43208\2035725282.py:6: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ref = df.groupby('SERIE').apply(lambda g: g.loc[g['FECHA'].idxmin(), col])
C:\Users\roquispec\AppData\Local\Temp\ipykernel_43208\2035725282.py:6: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ref = df.groupby('SERIE').apply(lambda g: g.loc[g['FECHA'].idxmin(), col])
C:\Users\roquispec\AppData\Local\Tem

,SERIE,FECHA,R POS1 mA,R POS2 mA,R POS3 mA,R POS4 mA,R POS5 mA,R POS6 mA,R POS7 mA,R POS8 mA,...,T POS18 mA_Delta,T POS19 mA_Delta,T POS20 mA_Delta,T POS21 mA_Delta,T POS22 mA_Delta,T POS23 mA_Delta,T POS24 mA_Delta,T POS25 mA_Delta,T POS26 mA_Delta,T POS27 mA_Delta
0,146660T3,2022-05-12,111.235,191.629,146.622,144.914,179.458,121.242,188.996,197.978,...,3.755280,58.013580,22.295739,21.894140,12.415228,18.640718,6.474012,9.363579,0.220207,13.264574
1,146660T3,2018-02-16,188.693,114.216,133.965,194.859,160.695,145.281,110.364,116.006,...,14.846144,90.766525,16.307387,11.311564,14.749621,3.804703,17.352133,19.073708,28.801473,54.359333
2,146660T3,2017-04-15,100.873,100.875,110.410,124.139,189.568,145.275,110.087,175.354,...,13.455259,13.630602,21.424706,24.390410,12.997306,47.265426,45.573911,3.691119,18.528088,7.498434
3,146660T3,2016-09-24,128.112,143.145,103.937,109.001,169.662,195.572,117.368,198.749,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,364076,2022-06-18,142.290,180.712,173.369,194.856,119.309,142.324,199.955,163.990,...,21.699776,31.188673,19.057075,59.535295,50.239764,14.542761,17.393660,15.738049,21.370257,4.137915
5,364076,2019-04-24,156.101,165.436,176.447,116.834,128.308,149.842,125.681,120.336,...,36.699070,11.678493,13.958229,46.208394,19.886675,0.346095,58.676186,39.837427,35.129460,15.781939
6,364076,2018-03-07,191.892,152.459,173.958,153.752,104.938,133.670,197.725,130.673,...,3.337226,7.170747,28.995682,18.965268,68.325765,21.930019,56.391553,21.914907,10.455416,19.462814
7,364076,2014-12-08,192.642,106.975,111.854,151.678,133.187,182.051,179.340,180.583,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,230531-01,2024-07-16,120.491,193.085,120.666,113.716,186.761,112.098,107.176,180.932,...,10.995503,26.684562,40.694898,24.147029,32.722088,13.112380,29.533661,35.743617,4.782891,12.617723
9,230531-01,2019-11-26,181.088,199.463,110.513,138.250,175.260,120.283,109.342,113.532,...,15.773593,28.265988,40.882708,27.166378,10.961353,1.271591,20.997092,43.179788,35.487958,7.801967


In [65]:
# Calcular el máximo de las columnas que terminan en 'Delta' para cada fila
cols_delta = [col for col in df.columns if col.endswith('Delta')]
df['Max_Iex'] = df[cols_delta].max(axis=1)
df.head()

,SERIE,FECHA,R POS1 mA,R POS2 mA,R POS3 mA,R POS4 mA,R POS5 mA,R POS6 mA,R POS7 mA,R POS8 mA,...,T POS19 mA_Delta,T POS20 mA_Delta,T POS21 mA_Delta,T POS22 mA_Delta,T POS23 mA_Delta,T POS24 mA_Delta,T POS25 mA_Delta,T POS26 mA_Delta,T POS27 mA_Delta,Max_Iex
0,146660T3,2022-05-12,111.235,191.629,146.622,144.914,179.458,121.242,188.996,197.978,...,58.013580,22.295739,21.894140,12.415228,18.640718,6.474012,9.363579,0.220207,13.264574,79.685365
1,146660T3,2018-02-16,188.693,114.216,133.965,194.859,160.695,145.281,110.364,116.006,...,90.766525,16.307387,11.311564,14.749621,3.804703,17.352133,19.073708,28.801473,54.359333,90.766525
2,146660T3,2017-04-15,100.873,100.875,110.410,124.139,189.568,145.275,110.087,175.354,...,13.630602,21.424706,24.390410,12.997306,47.265426,45.573911,3.691119,18.528088,7.498434,80.471952
3,146660T3,2016-09-24,128.112,143.145,103.937,109.001,169.662,195.572,117.368,198.749,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,364076,2022-06-18,142.290,180.712,173.369,194.856,119.309,142.324,199.955,163.990,...,31.188673,19.057075,59.535295,50.239764,14.542761,17.393660,15.738049,21.370257,4.137915,83.254186


In [66]:
# Asignar puntaje IEX según la tabla SD MYERS
# Si Max_Iex > 10% → 5, si Max_Iex ≤ 10% → 1
df['IEX'] = np.where(df['Max_Iex'] > 10, 5, 1)
df

,SERIE,FECHA,R POS1 mA,R POS2 mA,R POS3 mA,R POS4 mA,R POS5 mA,R POS6 mA,R POS7 mA,R POS8 mA,...,T POS20 mA_Delta,T POS21 mA_Delta,T POS22 mA_Delta,T POS23 mA_Delta,T POS24 mA_Delta,T POS25 mA_Delta,T POS26 mA_Delta,T POS27 mA_Delta,Max_Iex,IEX
0,146660T3,2022-05-12,111.235,191.629,146.622,144.914,179.458,121.242,188.996,197.978,...,22.295739,21.894140,12.415228,18.640718,6.474012,9.363579,0.220207,13.264574,79.685365,5
1,146660T3,2018-02-16,188.693,114.216,133.965,194.859,160.695,145.281,110.364,116.006,...,16.307387,11.311564,14.749621,3.804703,17.352133,19.073708,28.801473,54.359333,90.766525,5
2,146660T3,2017-04-15,100.873,100.875,110.410,124.139,189.568,145.275,110.087,175.354,...,21.424706,24.390410,12.997306,47.265426,45.573911,3.691119,18.528088,7.498434,80.471952,5
3,146660T3,2016-09-24,128.112,143.145,103.937,109.001,169.662,195.572,117.368,198.749,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1
4,364076,2022-06-18,142.290,180.712,173.369,194.856,119.309,142.324,199.955,163.990,...,19.057075,59.535295,50.239764,14.542761,17.393660,15.738049,21.370257,4.137915,83.254186,5
5,364076,2019-04-24,156.101,165.436,176.447,116.834,128.308,149.842,125.681,120.336,...,13.958229,46.208394,19.886675,0.346095,58.676186,39.837427,35.129460,15.781939,83.756325,5
6,364076,2018-03-07,191.892,152.459,173.958,153.752,104.938,133.670,197.725,130.673,...,28.995682,18.965268,68.325765,21.930019,56.391553,21.914907,10.455416,19.462814,85.982531,5
7,364076,2014-12-08,192.642,106.975,111.854,151.678,133.187,182.051,179.340,180.583,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1
8,230531-01,2024-07-16,120.491,193.085,120.666,113.716,186.761,112.098,107.176,180.932,...,40.694898,24.147029,32.722088,13.112380,29.533661,35.743617,4.782891,12.617723,96.068786,5
9,230531-01,2019-11-26,181.088,199.463,110.513,138.250,175.260,120.283,109.342,113.532,...,40.882708,27.166378,10.961353,1.271591,20.997092,43.179788,35.487958,7.801967,74.885548,5


In [67]:
# Crear tabla final solo con SERIE, FECHA, IEX y las columnas que no terminan en 'Delta' ni son 'Max_Iex'
cols_final = [col for col in df.columns if not col.endswith('Delta') and col != 'Max_Iex']
# Aseguramos el orden: SERIE, FECHA, IEX primero
cols_final = ['SERIE', 'FECHA', 'IEX'] + [col for col in cols_final if col not in ['SERIE', 'FECHA', 'IEX']]
df_final = df[cols_final]
df_final.head()

,SERIE,FECHA,IEX,R POS1 mA,R POS2 mA,R POS3 mA,R POS4 mA,R POS5 mA,R POS6 mA,R POS7 mA,...,T POS18 mA,T POS19 mA,T POS20 mA,T POS21 mA,T POS22 mA,T POS23 mA,T POS24 mA,T POS25 mA,T POS26 mA,T POS27 mA
0,146660T3,2022-05-12,5,111.235,191.629,146.622,144.914,179.458,121.242,188.996,...,186.193,160.812,120.611,114.643,143.999,132.651,176.578,161.892,147.003,124.753
1,146660T3,2018-02-16,5,188.693,114.216,133.965,194.859,160.695,145.281,110.364,...,152.812,194.145,129.906,163.382,188.661,107.555,156.040,119.796,104.434,170.016
2,146660T3,2017-04-15,5,100.873,100.875,110.410,124.139,189.568,145.275,110.087,...,155.308,115.643,121.963,182.579,143.042,164.656,102.757,142.567,173.857,118.402
3,146660T3,2016-09-24,1,128.112,143.145,103.937,109.001,169.662,195.572,117.368,...,179.454,101.771,155.218,146.779,164.411,111.809,188.801,148.031,146.680,110.143
4,364076,2022-06-18,5,142.290,180.712,173.369,194.856,119.309,142.324,199.955,...,163.629,121.602,137.777,198.430,153.521,186.329,136.476,145.228,190.737,160.036
